# Documentation and Versions of Tape Simulation System
[GitHub:] (https://github.com/sanchita-chakraborty/fermilab_simulating_tapes) 

## Version 1: Simple Queuing System (Completed: 6/9/23)
- Servers are like tellers in a bank
- Tapes work like individuals wanting to use bank services
- Features:
    - Time of arrival of tapes is automated using an inverse sampling formula!
    - Read time for tapes is max speed
    - Mounting speed is fixed
    - Data in server is fixed
    - Each Server can only support one type of tape

## Version 2: Queuing System: Server-Tape Interactions: (Completed 6/12/23)
- Tape characteristics assigned:
    - Tape address: randomized alphanumeric: Done
    - Tape type: randomly chosen for each between LTO8 and LTO9: Done
    - Tape speed: based on LTO8 or LTO9 characteristics: Done
    - Assigned random locations to servers and trays: Done
    - Data stored in tape - randomly assigned between (100,1000): Done
- Read tapes with 10-20% slowdown speed - random number generator of [0.8,0.9]: Done
- Write tapes with 10-20% slowdown speed - random number generator of [0.8,0.8]: Done
- Data is variable - accounted by a randon algorithm: Done
- Search and Sort Algorithm: Done
- Tape Server Relation Changes: DONE
    - Tape:
        - LTO8 and LTO9 characteristics need to be added to the tapes themselves: Done
        - Speed charactersitics need to be moved over to the tape charactersitics: Done
    - Server:
        - LTO8 and LTO9 characteristics need to be removed: DONE
        - Speed characteristics need to be removed: DONE
        - removed server specific service time generation methods since they were redundant: DONE
            - check if it is an IBM server which will need to add shuffle time: Done
- Search and Sort Algorithm Details: Done
    - Give a feeder tape to check for sample run
    - If address mathces, gives location
        - For IBM, does the shuffle algorithm
            - Shuffle algorithm is set to 1 second/frame to shuffle
    - Current Issue: Taking forever to run, even with 10 tapes: RESOLVED

## Version 3: User-Input and Non-Random Choices - (Completed 6/14/23)
- Characteristics Added:
    - Tape has a linear density array - tells you what the linear density of each tape is: Done
        - LTO8 Linear Density = 65620.8645647 bytes/inch
        - LTO9 Linear Density = 68132.2882086 bytes/inch
    - Array which holds the length of each file on a specific tape: Done
    - Pause time - time between files stored on tapes: Done
- Randomized choices changed to user input: Done
- Data pulled from actual sources instead of random numbers: Done
- Clever read/write mimicry algorithm: Done
    - Writing algorithm
        - randomly generates a tape to write to: Done
        - notes down the address and location of this tape: Done
        - randomly generates a file size and then turns it into a length of tape: Done
        - returns write time as well based on write_speed and file size: Done
        - will check if tape is full before writing a file to tape: Done
    - Reading algorithm
        - check the size of the file: Done
        - search for all files of that size as a preliminary check: Done
        - return reading speed of each file and add time appropriately: Done
        - once file found, return time: Done
        - Key Assumption: All files are compressed - affects read/write speed: Done
- Adjustements to arrival and departure algorithms: Done
    - Intention algorithm: is the queue item read/write action
        - first 4 tapes must be write - just for test purposes, as a file can't be read if there are no files stored: Done
- Key Issue: Service times are N/A for everything except for Server 2 - a bit strange. Need to see what is happening here. - FIXED

## Version 4: Realistic Assumptions, User Input, File Names and Robot Movement - Simulation COMPLETE (06/20/2023)
- Files written can be as large as the max data file storable on tape: Done
    - bias it towards smaller file size
- Check if file is compressed or not: Done
- Move the robot to location: Done
- Implement rewind featureL Done
- Implement 2 robots per server: Done
- Add LTO6 and LTO7: Done
- Implement File Names: Done 
- Link Addresses to File Names: Done
- Allow the User to specify how many LTO8, LTO9, LTO6 and LTO7 tapes are in each server: Done
- Allow the User to specify how many server trays are in each server: Done
- Allow the User to specify how many of each type of tape will be inputted.: Done

## Version 5: Areas of Improvement - INCOMPLETE
- find the closest file near the current file to write it - Implement method Find_Nearest_File(): Done
- implement a user input which tells you what server can take what kinds of tapes: Done
- Statistical analysis of data to implement proper mount and dismount times: Done
- Statistical analysis of data to implement proper read and write times: Done
- Monitor files that are accessed the most: Done
- move the files accessed the most to the front: Done
- move it to the server queue based on the address assigned

In [2]:
import numpy as np
import pandas as pd
import random as rand
import string as string
import math as math
import uuid

class Tape_Simulation:
    def __init__(self, s1type, s2type, s3type, s4type, s5type, server_trays1, server_trays2, server_trays3, server_trays4, server_trays5, numlto6, numlto7, numlto8, numlto9, compatible_tapes_S1, compatible_tapes_S2, compatible_tapes_S3, compatible_tapes_S4, compatible_tapes_S5):
        self.clock = 0.0 #simulation clock
        
        self.num_in_q = 0 #tape in queue
        self.t_shuffle = 0 #time to shuffle tape in IBM system
        self.t_load = 0 #time to load tape
        self.t_pause = 0.01 #time the tape reader pauses per file read - DEFAULT NUMBER
        self.t_mount = 0 #mount time of tape
        self.t_dismount = 0 #dismount time of tape
        self.total_wait_time = 0 #total wait time of each tape
        self.front_frame = 1
        self.write_speed = 1000 # MB/second
        self.tape_types_array = ['LTO8','LTO9','LTO6','LTO7']
        self.file_intent_array = ['read','write']
        self.file_data_format = ['compressed','uncompressed']
    
        self.num_in_system = 0
        self.number_in_queue = 0
        self.num_arrivals = 0 #number of arrivals
        self.num_departures = 0 #number of departures
        while True:
            self.t_arrival = (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3) #time of next arrival uing Inverse Transform Sampling
            if self.t_arrival > 0.0001:
                break
        self.slowdown = rand.uniform(0.6,0.7) # slowdown added to speeds for reading and writing 
        self.t_read_speed = 0 
        
        self.state_S1 = 0 #status of server (binary)
        self.num_of_departures1 = 0
        self.dep_sum_S1 = 0 #total departure time of server
        self.t_departure_S1 = float('inf') #departure time from server
        self.TYPE_S1 = s1type #type of storage server
        self.server_tray_assignment_S1 = server_trays1 #number of server trays in S1
        self.S1_compatible = compatible_tapes_S1 #will tell you what type of compatible tapes are for that server
        
        self.state_S2 = 0 #status of server (binary)
        self.num_of_departures2 = 0
        self.dep_sum_S2 = 0 #total departure time of server
        self.t_departure_S2 = float('inf') #departure time from server
        self.TYPE_S2 = s2type #type of storage server
        self.server_tray_assignment_S2 = server_trays2 #number of server trays in S2
        self.S2_compatible = compatible_tapes_S2 #will tell you what type of compatible tapes are for that server
        
        self.state_S3 = 0 #status of server (binary)
        self.num_of_departures3 = 0
        self.dep_sum_S3 = 0 #total departure time of server
        self.t_departure_S3 = float('inf') #departure time from server
        self.TYPE_S3 = s3type #type of storage server
        self.server_tray_assignment_S3 = server_trays3 #number of server trays in S3
        self.S3_compatible = compatible_tapes_S3 #will tell you what type of compatible tapes are for that server
        
        self.state_S4 = 0 #status of server (binary)
        self.num_of_departures4 = 0
        self.dep_sum_S4 = 0 #total departure time of server
        self.t_departure_S4 = float('inf') #departure time from server
        self.TYPE_S4 = s4type #type of storage server
        self.server_tray_assignment_S4 = server_trays4 #number of server trays in S4
        self.S4_compatible = compatible_tapes_S4 #will tell you what type of compatible tapes are for that server
        
        self.state_S5 = 0 #status of server (binary)
        self.num_of_departures5 = 0
        self.dep_sum_S5 = 0 #total departure time of server
        self.t_departure_S5 = float('inf') #departure time from server
        self.TYPE_S5 = s5type #type of storage server
        self.server_tray_assignment_S5 = server_trays5 #number of server trays in S5
        self.S5_compatible = compatible_tapes_S5 #will tell you what type of compatible tapes are for that server
        
        # robot allotted features
        self.robot_move_speed = 1 #second/tray - Based on youtube video observation
        self.robot_rotate_speed = 0.5 #second/rotation - Based on youtube video observation
        self.t_pull_tape = 1 #second - Based on youtube video observation
        self.t_push_tape = 1 #second - Based on youtube video observation
        self.address_old = [rand.randint(1,5),rand.randint(1,min(server_trays1, server_trays2, server_trays3, server_trays4, server_trays5)),rand.randint(1,5)] #initialize robot at a rand location
        if self.address_old[1] == 0 or self.address_old == 1 or self.address_old == 2 or self.address_old == 3 or self.address_old == 4:
            self.col_old = 1
        else:
            self.col_old = 2
        
        # define tape properties
        self.num_tapes = numlto6+numlto7+numlto8+numlto9
        self.num_lto6 = numlto6
        self.num_lto7 = numlto7
        self.num_lto8 = numlto8
        self.num_lto9 = numlto9
        self.num_files = 100
        self.file_names = ["" for x in range(self.num_files)] #empty array of names of files
        self.LTO6_storage_uncompressed = 2.5e6
        self.LTO6_storage_compressed = 6.25e6
        self.LTO7_storage_uncompressed = 6e6
        self.LTO7_storage_compressed = 1.5e7
        self.LTO8_storage_uncompressed = 1.2e7 #in MB
        self.LTO8_storage_compressed = 3.0e7 #in MB
        self.LTO9_storage_uncompressed = 1.8e7 #in MB
        self.LTO9_storage_compressed = 4.5e7 #in MB
        self.LTO8_choices = []
        self.LTO8_probability = [0.443928912, 0.214385019, 0.129930822, 0.065694042, 0.041573843, 0.033182024, 0.018676404,0.013065108, 0.011390094, 0.0121271, 0.008592821, 0.004522537, 0.001658264, 0.000686756,0.000150751, 0.000117251, 0.0000837507, 0.0000335003, 0.0000167501, 0.0000335003,0.0000335003, 0.0000502504, 0.0000167501, 0.0000167501, 0.0000335003]
        self.LTO9_choices = []
        self.LTO9_probability = [0.586803591, 0.187841227, 0.097179954, 0.055754791, 0.028586612, 0.013155882, 0.006939241,0.005214944, 0.005497867, 0.005394638, 0.003926501, 0.002240438, 0.000860236, 0.000340271,0.0000917586, 0.0000191164, 0.0000191164, 0.00000764655, 0.00000764655, 0.0000114698,0.0000191164, 0.00000764655, 0.00000764655, 0.0000114698, 0.00000382327, 0.00000382327, 0.0000229396, 0.0000191164, 0.0000114698]
        self.LTO9_choices_dismount = []
        self.LTO9_probability_dismount = [0.639880998, 0.153925383, 0.086816351, 0.051862374, 0.027169834, 0.012118451, 0.00633682,0.004893327, 0.00567825, 0.005146034, 0.003377085, 0.001692371, 0.000616452, 0.000183787,0.0000536045, 0.0000650912, 0.0000191445, 0.0000153156, 0.0000153156, 0.0000153156,0.0000306312, 0.0000191445, 0.0000114867, 0.0000229734, 0.00000765779, 0.00000765779,0.0000114867, 0.00000382889, 0.00000382889]
        self.LTO8_choices_dismount = []
        self.LTO8_probability_dismount = [0.536726005, 0.188591639, 0.094790305, 0.049096517, 0.038234604, 0.027456502, 0.016142009,0.012404036, 0.01250461, 0.011180395, 0.007291562, 0.00333568, 0.00129069, 0.00045258,0.000217909, 0.0000670488, 0.0000502866, 0.0000167622, 0.0000167622, 0.0000335244,0.0000167622, 0.0000335244, 0.0000167622, 0.0000335244]
        self.mount_array()
        self.dismount_array()
        
        #ordered properly 
        self.tape_type = ["" for x in range(self.num_tapes)]
        self.tape_data_stored = [0 for x in range(self.num_tapes)]
        self.assignment = [[0]*3 for m in range(self.num_tapes)]
        self.tape_address_len = 16 #length of address
        self.tape_linear_density = [0 for x in range(self.num_tapes)];
        self.tape_addresses = ["" for x in range(self.num_tapes)] #array of empty addresses for tapes
        
        self.file_intent = ["" for x in range(self.num_files)] #array of empty addresses for tapes
        self.file_data_type = ["" for x in range(self.num_files)] #array of compressed/uncompressed for tapes
        self.tape_files = [[-1 for i in range(self.num_files)] for j in range(self.num_tapes)] #array which will hold the length of each file on tape
        self.file_access = [[0 for i in range(self.num_files)] for j in range(self.num_tapes)] #array which will hold the number of times files have been accessed 
        self.assign_tape_loc()
        feeder_address = self.tape_addresses[rand.randint(0,len(self.tape_addresses)-1)]
        self.search_sort()
    
    def robot_move(self, action, move_dist):
        #do something to move robot
        if action == "move+rotate":
            move_time = self.robot_move_speed*move_dist
            rotate_time = 1
        elif action == "move":
            move_time = self.robot_move_speed*move_dist
            rotate_time = 0
        elif action == "rotate":
            rotate_time = 1
            move_time = 0
        else:
            rotate_time = 0
            move_time = 0
        return (move_time+rotate_time)
    
    def mount_array(self):
        start = 5; i = 0;
        while i < len(self.LTO9_probability):
            self.LTO9_choices.append(rand.randrange(start,start+99))
            start += 100
            i+=1; 
        start = 5; i = 0;
        while i < len(self.LTO8_probability):
            self.LTO8_choices.append(rand.randrange(start,start+99))
            start += 100
            i+=1
        print(self.LTO8_choices,self.LTO8_probability)
        print(len(self.LTO8_choices),len(self.LTO8_probability))
        print(self.LTO9_choices,self.LTO9_probability)
        print(len(self.LTO9_choices),len(self.LTO9_probability))
    
    def dismount_array(self):
        start = 7; i = 0;
        while i < len(self.LTO9_probability_dismount):
            self.LTO9_choices_dismount.append(rand.randrange(start,start+99))
            start += 100
            i+=1
        start = 7; i = 0
        while i < len(self.LTO8_probability_dismount):
            self.LTO8_choices_dismount.append(rand.randrange(start,start+99))
            start += 100
            i+=1
        print(self.LTO8_choices_dismount,self.LTO8_probability_dismount)
        print(len(self.LTO8_choices_dismount),len(self.LTO8_probability_dismount))
        print(self.LTO9_choices_dismount,self.LTO9_probability_dismount)
        print(len(self.LTO9_choices_dismount),len(self.LTO9_probability_dismount))
        
    def mount(self,tape_number):
        mount_time_LTO9 = rand.choices(self.LTO9_choices, weights=self.LTO9_probability, k=1)
        mount_time_LTO8 = rand.choices(self.LTO8_choices, weights=self.LTO8_probability, k=1)
        mount_delay_time = 15
        if self.tape_type[tape_number] == "LTO6" or self.tape_type[tape_number] == "LTO7" or self.tape_type[tape_number] == "LTO8":
            mount_time = mount_time_LTO8[0]+mount_delay_time
        else:
            mount_time = mount_time_LTO9[0]+mount_delay_time
        return mount_time
        
    def dismount(self,tape_number):
        dismount_time_LTO9 = rand.choices(self.LTO9_choices_dismount, weights=self.LTO9_probability_dismount, k=1)
        dismount_time_LTO8 = rand.choices(self.LTO8_choices_dismount, weights=self.LTO8_probability_dismount, k=1)
        dismount_delay_time = 15
        if self.tape_type[tape_number] == "LTO6" or self.tape_type[tape_number] == "LTO7" or self.tape_type[tape_number] == "LTO8":
            dismount_time = dismount_time_LTO8[0]+dismount_delay_time
        else:
            dismount_time = dismount_time_LTO9[0]+dismount_delay_time
        return dismount_time
        
    def sort_usage(self):
        #tracks usage of each tape
        tape_sorter = [0 for x in range(self.num_files)]
        for count_tape in range(self.num_tapes):
            for count_file in range(self.num_files):
                tape_sorter[count_tape] += self.file_access[count_tape][count_file]
        #actual sorting algorithm
        for i in range(self.num_tapes):
            for j in range(0, self.num_tapes - i - 1):
                # Range of the array is from 0 to n-i-1
                # Swap the elements if the element found
                #is greater than the adjacent element
                if tape_sorter[j] > tape_sorter[j + 1]:
                    tape_sorter[j], tape_sorter[j + 1] = tape_sorter[j + 1], tape_sorter[j] #flip usage entries
                    self.tape_addresses[j], self.tape_addresses[j + 1] = self.tape_addresses[j + 1], self.tape_addresses[j] #flip tape addresses
                    self.tape_type[j], self.tape_type[j+1] = self.tape_type[j+1], self.tape_type[j] #flip tape types
                    self.tape_data_stored[j], self.tape_data_stored[j+1] = self.tape_data_stored[j+1],self.tape_data_stored[j] 
                    self.assignment[j], self.assignment[j+1] = self.assignment[j+1], self.assignment[j]
                    self.tape_linear_density[j], self.tape_linear_density[j+1] = self.tape_linear_density[j+1], self.tape_linear_density[j]
            
    # Algorithms involving interactions with tapes: read, write, open, and close
    def read(self, file_size, file_name):
        # Check - see if the size matches any pre-existing file size
        read_time = 0
        address_new = self.address_old
        col_new = 0
        mount_time = 0; dismount_time = 0;
        for count_tape in range(self.num_tapes):
            details = self.assignment[count_tape]
            reading_speed = details[3]
            for count_file in range(self.num_files):
                #read_time += self.tape_files[count_tape][count_file]/reading_speed
                mount_time = self.mount(count_tape); dismount_time = self.dismount(count_tape)
                self.file_access[count_tape][count_file] += 1
                if self.tape_files[count_tape][count_file] == file_size or self.file_names[count_file] == file_name:
                    address_new = self.assignment[count_tape]
                read_time += self.t_pause
                #Data based read-time
                if address_new[0] == 1 or address_new[0] == 2 or address_new[0] == 3 or address_new[0] == 4:
                    read_time += np.random.normal(loc=1566.120081, scale = 4878.121746, size=1)
                else:
                    read_time += np.random.normal(loc=2387.482149, scale = 2671.137441, size=1)
                if address_new[0] == self.address_old[0]:
                    if address_new[1] == self.address_old[1]+5 or address_new[1] == self.address_old[1]-5:
                        action = "rotate"
                        move_amt = 0
                        if address_new[1] == 0 or address_new[1] == 1 or address_new[1] == 2 or address_new[1] == 3 or address_new[1] == 4:
                            col_new = 1
                        else:
                            col_new = 2
                    else:
                        if address_new[1] == 0 or address_new[1] == 1 or address_new[1] == 2 or address_new[1] == 3 or address_new[1] == 4:
                            col_new = 1
                        else:
                            col_new = 2
                        if col_new == self.col_old:
                            action = "move"
                            move_amt = abs(address_new[1]-self.address_old[1])
                        else:
                            if col_new == 2:
                                temp_tray = address_new[1]-5
                                move_amt = abs(temp_tray-self.address_old[1])
                            else:
                                temp_tray = address_new[1]-5
                                move_amt = abs(temp_tray-self.address_old[1])
                            action = "move+rotate"
                else:
                    action = "do nothing"
                    move_amt = 0
        if action == "move+rotate":
            move_time = self.robot_move_speed*move_amt
            rotate_time = 1
        elif action == "move":
            move_time = self.robot_move_speed*move_amt
            rotate_time = 0
        elif action == "rotate":
            rotate_time = 1
            move_time = 0
        else:
            rotate_time = 0
            move_time = 0
        robot_time = move_time+rotate_time
        self.address_old = address_new
        self.col_old = col_new
        self.sort_usage()
        return read_time+robot_time+mount_time+dismount_time
    
    def write(self):
        output = ""
        write_time = 0; mount_time = 0; dismount_time = 0;
        indices = [0,0]
        address_new = self.address_old
        col_new = 0
        # pick a random tape to write to
        tape_choice = rand.choice(self.assignment)
        
        # find the address of this random tape - [server, server_tray, IBM layer or 0]
        tape_counter = self.assignment.index(tape_choice)
        numrows = len(self.assignment)
        numcols = len(self.assignment[1])
        count = 0
        for i in range(numrows):
            for j in range(numcols):
                if self.assignment[i][j] == -1 and count == 0:
                    indices = [i,j]
                    count += 1
                else:
                    count+=0
        
        file_counter = indices[1]
        address = self.assignment[tape_counter]
        #Data based write-time
        if address[0] == 1:
            write_time += np.random.normal(loc=5.603174718, scale = 18.09117107, size=1)
        elif address[0] == 2:
            write_time += np.random.normal(loc=15.94541094, scale = 21.19278628, size=1)
        elif address[0] == 3:
            write_time += np.random.normal(loc=10284.92346, scale = 1240.03684, size=1)
        elif address[0] == 4:
            write_time += np.random.normal(loc=681.6435276, scale = 1192.43544, size=1)
        else:
            write_time += np.random.normal(loc=4299.56154, scale = 3193.226176, size=1)
        mount_time = self.mount(tape_counter); dismount_time = self.dismount(tape_counter)
        
        # add write time for reading and checking that other file slots in the length are full
        #if file_counter != 0:
            #for count_dispose2 in range(tape_counter):
                #for count_dispose in range(file_counter):
                    #details = self.assignment[count_dispose2]
                    #read_speed = details[3]
                    #write_time += self.tape_files[count_dispose2][count_dispose]/read_speed
                #address_new = self.assignment[count_dispose2]
        
        if address_new[0] == 1:
            shift = math.floor(self.server_tray_assignment_S1/2)
        elif address_new[0] == 2:
            shift = math.floor(self.server_tray_assignment_S2/2)
        elif address_new[0] == 3:
            shift = math.floor(self.server_tray_assignment_S3/2)
        elif address_new[0] == 4:
            shift = math.floor(self.server_tray_assignment_S4/2)
        else:
            shift = math.floor(self.server_tray_assignment_S5/2)
        # randomly generate data for each file
        if address_new[0] == self.address_old[0]:
            if address_new[1] == self.address_old[1]+shift or address_new[1] == self.address_old[1]-shift:
                action = "rotate"
                move_amt = 0
                if address_new[1] == 0 or address_new[1] == 1 or address_new[1] == 2 or address_new[1] == 3 or address_new[1] == 4:
                    col_new = 1
                else:
                    col_new = 2
            else:
                if address_new[1] <= shift-1:
                    col_new = 1
                else:
                    col_new = 2
                if col_new == self.col_old:
                    action = "move"
                    move_amt = abs(address_new[1]-self.address_old[1])
                else:
                    if col_new == 2:
                        temp_tray = address_new[1]-5
                        move_amt = abs(temp_tray-self.address_old[1])
                    else:
                        temp_tray = address_new[1]-5
                        move_amt = abs(temp_tray-self.address_old[1])
                    action = "move+rotate"
        else:
            action = "do nothing"
            move_amt = 0
        
        if action == "move+rotate":
            move_time = self.robot_move_speed*move_amt
            rotate_time = 1
        elif action == "move":
            move_time = self.robot_move_speed*move_amt
            rotate_time = 0
        elif action == "rotate":
            rotate_time = 1
            move_time = 0
        else:
            rotate_time = 0
            move_time = 0
        robot_time = move_time+rotate_time
        
        write_time = 0
        file_data = 20000*(1 - math.sqrt(1 - np.random.uniform(low=0.0,high=1.0))) #randomly generate size of file for each write event (in MBs) between 1 MBs to 20 GBs
        while True:
            if self.check_tape_storage(tape_counter) == 0:
                tape_file_length = file_data/self.tape_linear_density[tape_counter]
                break
            else:
                file_data = 20000*(1 - math.sqrt(1 - np.random.uniform(low=0.0,high=1.0)))
        if file_counter != self.num_files:
            self.tape_files[tape_counter][file_counter] = tape_file_length
            self.file_names[file_counter] = str(uuid.uuid4())
            if file_counter != self.num_files-1:
                self.tape_files[tape_counter][file_counter+1] = -2
        else:
            output = 'Too full! Need to initiate different tape sequence'
            self.find_nearest_file(tape_count,file_count, file_name,"write")
        write_time += (tape_file_length/self.write_speed)+robot_time+mount_time+dismount_time
        self.address_old = address_new
        self.col_old = col_new
        return [write_time, output]
    
    def check_tape_storage(self,tape_number):
        storage = 0
        for i in range(self.num_files):
            if self.tape_files[tape_number][i] != -2 and self.tape_files[tape_number][i] != -1:
                storage += self.tape_files[tape_number][i]
            else:
                storage += 0
        if self.tape_type[tape_number] == "LTO6" and self.file_data_type[tape_number] == 'uncompressed':
            if storage > self.LTO6_storage_uncompressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO6" and self.file_data_type[tape_number] == 'compressed':
            if storage > self.LTO6_storage_compressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO7" and self.file_data_type[tape_number] == 'uncompressed':
            if storage > self.LTO7_storage_uncompressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO7" and self.file_data_type[tape_number] == 'compressed':
            if storage > self.LTO7_storage_compressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO8" and self.file_data_type[tape_number] == 'uncompressed':
            if storage > self.LTO8_storage_uncompressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO8" and self.file_data_type[tape_number] == 'compressed':
            if storage > self.LTO8_storage_compressed:
                return -1
            else:
                return 0
        elif self.tape_type[tape_number] == "LTO9" and self.file_data_type[tape_number] == 'uncompressed':
            if storage > self.LTO9_storage_uncompressed:
                return -1
            else:
                return 0
        else:
            if storage > self.LTO9_storage_compressed:
                return -1
            else:
                return 0
    
    # simple search and sort algorithm for Layer
    def search_sort(self):
        feeder_address = self.tape_addresses[rand.randint(0,len(self.tape_addresses)-1)]
        for k in range(self.num_tapes):
            if feeder_address == self.tape_addresses[k]:
                location = self.assignment[k]
                if location[2] == 3:
                    if self.IBM_layer != 0 and self.IBM_layer != self.front_frame:
                        self.shuffle()
            else:
                location = -1
                self.t_shuffle = 0
        return location
    
    def compatibility_servers(self):
        LTO6_compatible_servers = []; LTO7_compatible_servers = []; LTO8_compatible_servers = []; LTO9_compatible_servers = [];
        if "LTO6" in self.S1_compatible:
            LTO6_compatible_servers.append(1)
        if "LTO7" in self.S1_compatible:
            LTO7_compatible_servers.append(1)
        if "LTO8" in self.S1_compatible:
            LTO8_compatible_servers.append(1)
        if "LTO9" in self.S1_compatible:
            LTO9_compatible_servers.append(1)
        if "LTO6" in self.S2_compatible:
            LTO6_compatible_servers.append(2)
        if "LTO7" in self.S2_compatible:
            LTO7_compatible_servers.append(2)
        if "LTO8" in self.S2_compatible:
            LTO8_compatible_servers.append(2)
        if "LTO9" in self.S2_compatible:
            LTO9_compatible_servers.append(2)
        if "LTO6" in self.S3_compatible:
            LTO6_compatible_servers.append(3)
        if "LTO7" in self.S3_compatible:
            LTO7_compatible_servers.append(3)
        if "LTO8" in self.S3_compatible:
            LTO8_compatible_servers.append(3)
        if "LTO9" in self.S3_compatible:
            LTO9_compatible_servers.append(3)
        if "LTO6" in self.S4_compatible:
            LTO6_compatible_servers.append(4)
        if "LTO7" in self.S4_compatible:
            LTO7_compatible_servers.append(4)
        if "LTO8" in self.S4_compatible:
            LTO8_compatible_servers.append(4)
        if "LTO9" in self.S4_compatible:
            LTO9_compatible_servers.append(4)
        if "LTO6" in self.S5_compatible:
            LTO6_compatible_servers.append(5)
        if "LTO7" in self.S5_compatible:
            LTO7_compatible_servers.append(5)
        if "LTO8" in self.S5_compatible:
            LTO8_compatible_servers.append(5)
        if "LTO9" in self.S5_compatible:
            LTO9_compatible_servers.append(5)
        return [LTO6_compatible_servers,LTO7_compatible_servers,LTO8_compatible_servers,LTO9_compatible_servers]
    
    #assign location to all tapes
    def assign_tape_loc(self):
        count_lto6 = 0
        count_lto7 = 0
        count_lto8 = 0
        count_lto9 = 0
        [lto6_servers, lto7_servers, lto8_servers, lto9_servers] = self.compatibility_servers()
        for j in range(self.num_tapes):
            self.tape_addresses[j] = ''.join(rand.choices(string.ascii_letters + string.digits, k=self.tape_address_len)) #random alphanumeric addresses for tapes of specified length
            choice_array = self.tape_types_array
            if j == 1:
                self.tape_type[j] = rand.choice(self.tape_types_array)
            else:
                if count_lto6 >= self.num_lto6 and 'LTO6' in choice_array:
                    choice_array.remove('LTO6')
                if count_lto7 >= self.num_lto7 and 'LTO7' in choice_array:
                    choice_array.remove('LTO7')
                if count_lto8 >= self.num_lto8 and 'LTO8' in choice_array:
                    choice_array.remove('LTO8')
                if count_lto9 >= self.num_lto9 and 'LTO9' in choice_array:
                    choice_array.remove('LTO9')
                self.tape_type[j] = rand.choice(choice_array)
                if self.tape_type[j] == 'LTO6':
                    count_lto6 += 1
                elif self.tape_type[j] == 'LTO7':
                    count_lto7 += 1
                elif self.tape_type[j] == 'LTO8':
                    count_lto8 += 1
                else:
                    count_lto9 += 1
            for j2 in range(self.num_files):
                self.file_data_type[j2] = rand.choice(self.file_data_format)
                if j2 >= 0 and j2 < 3:
                    self.file_intent[j2] = 'write'
                else:
                    self.file_intent[j2] = rand.choice(self.file_intent_array)
            if self.tape_type[j] == 'LTO6':
                self.tape_linear_density[j] = 48078.9990373 #linear density in bytes/inch for LTO6
                self.server_assignment = rand.choice(lto6_servers) #generates a random server to be assigned to for each tape
            elif self.tape_type[j] == 'LTO7':
                self.tape_linear_density[j] = 60623.4172634 #linear density in bytes/inch for LTO7
                self.server_assignment = rand.choice(lto7_servers) #generates a random server to be assigned to for each tape
            elif self.tape_type[j] == 'LTO8':
                self.tape_linear_density[j] = 65620.8645647 #linear density in bytes/inch for LTO8
                self.server_assignment = rand.choice(lto8_servers) #generates a random server to be assigned to for each tape
            else:
                self.tape_linear_density[j] = 68132.2882086 #linear density in bytes/inch for LTO9
                self.server_assignment = rand.choice(lto9_servers) #generates a random server to be assigned to for each tape
            if self.server_assignment == 1:
                self.server_tray_assignment_S1 = rand.randint(1,self.server_tray_assignment_S1)
                if self.TYPE_S1 == 'IBM':
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
            elif self.server_assignment == 2:
                self.server_tray_assignment_S2 = rand.randint(1,self.server_tray_assignment_S2)
                if self.TYPE_S2 == 'IBM':
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
            
            elif self.server_assignment == 3:
                self.server_tray_assignment_S3 = rand.randint(1,self.server_tray_assignment_S3)
                if self.TYPE_S3 == 'IBM':
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
            elif self.server_assignment == 4:
                self.server_tray_assignment_S4 = rand.randint(1,self.server_tray_assignment_S4)
                if self.TYPE_S4 == 'IBM':
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
            else:
                self.server_tray_assignment_S5 = rand.randint(1,self.server_tray_assignment_S5)
                if self.TYPE_S5 == 'IBM':
                    self.IBM_layer = rand.randint(1,5)
                else:
                    self.IBM_layer = 0
            self.tape_data_stored[j] = rand.randint(100,1000)
            
            if self.tape_type[j] == 'LTO6':
                self.slowdown = rand.uniform(0.6,0.7) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 640*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 640*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 640*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 640*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 640*self.slowdown #speed to read tape
            elif self.tape_type[j] == 'LTO7':
                self.slowdown = rand.uniform(0.6,0.7) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 750*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 750*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 750*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 750*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 750*self.slowdown #speed to read tape
            elif self.tape_type[j] == 'LTO8':
                self.slowdown = rand.uniform(0.6,0.7) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 900*self.slowdown #speed to read tape
            else: #LTO9
                self.slowdown = rand.uniform(0.6,0.7) # slowdown added to speeds for reading and writing 
                if self.server_assignment == 1:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 2:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 3:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                elif self.server_assignment == 4:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
                else:
                    self.t_read_speed = 1000*self.slowdown #speed to read tape
            
            if self.server_assignment == 1:
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment_S1, self.IBM_layer, self.t_read_speed]
            elif self.server_assignment == 2:
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment_S2, self.IBM_layer, self.t_read_speed]
            elif self.server_assignment == 3:
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment_S3, self.IBM_layer, self.t_read_speed]
            elif self.server_assignment == 4:
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment_S4, self.IBM_layer, self.t_read_speed]
            else:
                self.assignment[j] = [self.server_assignment, self.server_tray_assignment_S5, self.IBM_layer, self.t_read_speed]
        return self.assignment
    
    #finds closest open spot
    def find_nearest_file(self,tape_count,file_count, file_name,rw_action):
        if self.new_address[2] > self.old_address[2]+1 or self.new_address[2] < self.old_address[2]-1:
            if rw_action == read:
                self.read(file_size, file_name)
            else:
                self.write()
        else:
            i = rand.choice([-1, 1])
            count_full = 0
            while True:
                self.new_address[1] = self.old_address[1]+i #moves over to the next tape
                [time,out] = self.write()
                if out != 'Too full! Need to initiate different tape sequence':
                    break
    
    #timing routine       
    def time_adv(self,tape_count,file_count,file_name):
        if self.t_departure_S1 != float('inf'):
            t_next_event = self.t_departure_S1
        elif self.t_departure_S2 != float('inf'):
            t_next_event = self.t_departure_S2
        elif self.t_departure_S3 != float('inf'):
            t_next_event = self.t_departure_S3
        elif self.t_departure_S4 != float('inf'):
            t_next_event = self.t_departure_S4
        elif self.t_departure_S5 != float('inf'):
            t_next_event = self.t_departure_S5
        else:
            t_next_event = 0
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock = t_next_event
        
        if self.t_arrival < self.t_departure_S1 and self.t_arrival < self.t_departure_S2 and self.t_arrival < self.t_departure_S3 and self.t_arrival < self.t_departure_S4 and self.t_arrival < self.t_departure_S5:
            self.arrival(tape_count,file_count,file_name)
        elif self.t_departure_S1 < self.t_arrival and self.t_departure_S1 < self.t_departure_S2 and self.t_departure_S1 < self.t_departure_S3 and self.t_departure_S1 < self.t_departure_S4 and self.t_departure_S1 < self.t_departure_S5:
            self.S1(tape_count,file_count,file_name)
        elif self.t_departure_S2 < self.t_arrival and self.t_departure_S2 < self.t_departure_S1 and self.t_departure_S2 < self.t_departure_S3 and self.t_departure_S2 < self.t_departure_S4 and self.t_departure_S2 < self.t_departure_S5:
            self.S2(tape_count,file_count,file_name)
        elif self.t_departure_S3 < self.t_arrival and self.t_departure_S3 < self.t_departure_S1 and self.t_departure_S3 < self.t_departure_S2 and self.t_departure_S3 < self.t_departure_S4 and self.t_departure_S3 < self.t_departure_S5:
            self.S3(tape_count,file_count,file_name)
        elif self.t_departure_S4 < self.t_arrival and self.t_departure_S4 < self.t_departure_S1 and self.t_departure_S4 < self.t_departure_S2 and self.t_departure_S4 < self.t_departure_S3 and self.t_departure_S4 < self.t_departure_S5:
            self.S4(tape_count,file_count,file_name)
        else:
            self.S5(tape_count,file_count,file_name)
    
    def shuffle(self):
        if self.IBM_layer == self.front_frame+1 or self.IBM_layer == self.front_frame-1:
            self.t_shuffle = 1
        elif self.IBM_layer == self.front_frame+2 or self.IBM_layer == self.front_frame-2:
            self.t_shuffle = 2
        elif self.IBM_layer == self.front_frame+3 or self.IBM_layer == self.front_frame-3:
            self.t_shuffle = 3
        else:
            self.t_shuffle = 4
        
        self.front_frame = self.IBM_layer
        return self.t_shuffle

    #arrival event
    def arrival(self,tape_count,file_count,file_name):
        self.num_arrivals += 1
        self.num_in_system += 1
        count = 0
        status_servers = [self.state_S1, self.state_S2, self.state_S3, self.state_S4, self.state_S5]
        if self.num_in_q == 0:
            if all(element == 2 for element in status_servers):
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival = self.clock+self.t_arrival
            elif all(element == 0 for element in status_servers) or all(element == 1 for element in status_servers):
                if rand.randint(1,5) == 1:
                    self.state_S1 = 1
                    self.dep1= self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + self.t_arrival
                elif rand.randint(1,5) == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + self.t_arrival
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + self.t_arrival
            else:
                for status in range(0,4):
                    if status_servers[status] == 1:
                        count+=1
                if count != 0:
                    seed = rand.randint(1,count)
                else:
                    seed = 0
                if seed == 1:
                    self.state_S1 = 1
                    self.dep1 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S1 += self.dep1
                    self.t_departure_S1 = self.clock + self.dep1
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 2:
                    self.state_S2 = 1
                    self.dep2 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S2 += self.dep2
                    self.t_departure_S2 = self.clock + self.dep2
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 3:
                    self.state_S3 = 1
                    self.dep3 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S3 += self.dep3
                    self.t_departure_S3 = self.clock + self.dep3
                    self.t_arrival = self.clock + self.t_arrival
                elif seed == 4:
                    self.state_S4 = 1
                    self.dep4 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S4 += self.dep4
                    self.t_departure_S4 = self.clock + self.dep4
                    self.t_arrival = self.clock + self.t_arrival
                else:
                    self.state_S5 = 1
                    self.dep5 = self.gen_service_time(tape_count,file_count,file_name)
                    self.dep_sum_S5 += self.dep5
                    self.t_departure_S5 = self.clock + self.dep5
                    self.t_arrival = self.clock + self.t_arrival 
        else:
            self.num_in_q += 1
            self.number_in_queue += 1
            self.t_arrival = self.clock + self.t_arrival
    
    def S1(self,tape_count,file_count,file_name):
        self.num_of_departures1 += 1
        if self.num_in_q > 0:
            self.dep1 = self.gen_service_time(tape_count,file_count,file_name)
            self.dep_sum_S1 += self.dep1
            self.t_departure_S1 = self.clock + self.dep1
            self.num_in_q -= 1
        else:
            self.t_departure_S1 = float('inf')
            self.state_S1 = 0
    
    def S2(self,tape_count,file_count,file_name):
        self.num_of_departures2 += 1
        if self.num_in_q > 0:
            self.dep2 = self.gen_service_time(tape_count,file_count,file_name)
            self.dep_sum_S2 += self.dep2
            self.t_departure_S2 = self.clock + self.dep2
            self.num_in_q -= 1
        else:
            self.t_departure_S2 = float('inf')
            self.state_S2 = 0
    
    def S3(self,tape_count,file_count,file_name):
        self.num_of_departures3 += 1
        if self.num_in_q > 0:
            self.dep3 = self.gen_service_time(tape_count,file_count,file_name)
            self.dep_sum_S3 += self.dep3
            self.t_departure_S3 = self.clock + self.dep3
            self.num_in_q -= 1
        else:
            self.t_departure_S3 = float('inf')
            self.state_S3 = 0
    
    def S4(self,tape_count,file_count,file_name):
        self.num_of_departures4 += 1
        if self.num_in_q > 0:
            self.dep4 = self.gen_service_time(tape_count,file_count,file_name)
            self.dep_sum_S4 += self.dep4
            self.t_departure_S4 = self.clock + self.dep4
            self.num_in_q -= 1
        else:
            self.t_departure_S4 = float('inf')
            self.state_S4 = 0
    
    def S5(self,tape_count,file_count,file_name):
        self.num_of_departures5 += 1
        if self.num_in_q > 0:
            self.dep5 = self.gen_service_time(tape_count,file_count,file_name)
            self.dep_sum_S5 += self.dep5
            self.t_departure_S5 = self.clock + self.dep5
            self.num_in_q -= 1
        else:
            self.t_departure_S5 = float('inf')
            self.state_S5 = 0
    
    #Generate Arrival and Service Time from Servers
    def type_check(self,tape_counter,file_counter,type_of_server,action,file_name):
        read_time = 0
        write_time = 0
        if type_of_server == 'IBM':
            if self.tape_type[tape_counter] == 'LTO6':
                load_to_ready_time = 12
                if self.tape_files[tape_counter][file_counter] < 5000:
                    rewind_time = 55
                elif self.tape_files[tape_counter][file_counter] >= 5000 and self.tape_files[tape_no][count_file] < 50000:
                    rewind_time = 110
                else:
                    rewind_time = 165
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            
            elif self.tape_type[tape_counter] == 'LTO7':
                load_to_ready_time = 12
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            elif self.tape_type[tape_counter] == 'LTO8':
                load_to_ready_time = 15
                rewind_time = 60
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            else:
                load_to_ready_time = 17
                rewind_time = 60
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            return (load_to_ready_time+rewind_time+read_time+write_time+self.t_shuffle)
        else:
            if self.tape_type[tape_counter] == 'LTO6':
                load_to_ready_time = 12
                rewind_time = 0
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            elif self.tape_type[tape_counter] == 'LTO7':
                load_to_ready_time = 12
                rewind_time = 0
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            elif self.tape_type[tape_counter] == 'LTO8':
                load_to_ready_time = 15
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            else:
                load_to_ready_time = 17
                rewind_time = 59
                if action[file_counter] == 'read':
                    read_time += self.read(self.tape_files[tape_counter][file_counter],file_name)
                else:
                    [time,err] = self.write()
                    write_time += time
            return (load_to_ready_time+rewind_time+read_time+write_time)
    
    def gen_service_time(self,tape_counter,file_counter,file_name):
        address = self.assignment[tape_counter]
        server_number = address[0]
        if server_number == 1:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S1,self.file_intent,file_name)
        elif server_number == 2:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S2,self.file_intent,file_name)
        elif server_number == 3:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S3,self.file_intent,file_name)
        elif server_number == 4:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S4,self.file_intent,file_name)
        else:
            time = self.type_check(tape_counter,file_counter,self.TYPE_S5,self.file_intent,file_name)
        return time


# RUN - Main Part

# Change server types
S1_type = "inv"
S2_type = "inv"
S3_type = "inv"
S4_type = "inv"
S5_type = "inv"

type_input_S1 = input('Enter whether IBM or NonIBM for Server 1 ')
type_input_server_trays_S1 = int(input('Enter how many server trays are in Server 1 '))
while S1_type == "inv":
    if type_input_S1.upper() == "NonIBM".upper() or type_input_S1.upper() == "Non IBM".upper():
        S1_type = 'NonIBM'
        input_tape_type_S1 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S1 = input_tape_type_S1.split(",")
    elif type_input_S1.upper() == "IBM".upper():
        S1_type = 'IBM'
        input_tape_type_S1 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S1 = input_tape_type_S1.split(",")
    else:
        S1_type = "inv"
        print('Invalid response, respond as Non IBM or IBM ')
        type_input_S1 = input('Enter whether IBM or NonIBM for Server 1 ')
        
type_input_S2 = input('Enter whether IBM or NonIBM for Server 2 ')
type_input_server_trays_S2 = int(input('Enter how many server trays are in Server 2 '))
while S2_type == "inv":
    if type_input_S2.upper() == "NonIBM".upper() or type_input_S2.upper() == "Non IBM".upper():
        S2_type = 'NonIBM'
        input_tape_type_S2 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S2 = input_tape_type_S2.split(",")
    elif type_input_S2.upper() == "IBM".upper():
        S2_type = 'IBM'
        input_tape_type_S2 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S2 = input_tape_type_S2.split(",")
    else:
        S2_type = "inv"
        print('Invalid response, respond as Non IBM or IBM ')
        type_input_S2 = input('Enter whether IBM or NonIBM for Server 2 ')
        
type_input_S3 = input('Enter whether IBM or NonIBM for Server 3 ')
type_input_server_trays_S3 = int(input('Enter how many server trays are in Server 3 '))
while S3_type == "inv":
    if type_input_S3.upper() == "NonIBM".upper() or type_input_S3.upper() == "Non IBM".upper():
        S3_type = 'NonIBM'
        input_tape_type_S3 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S3 = input_tape_type_S3.split(",")
    elif type_input_S3.upper() == "IBM".upper():
        S3_type = 'IBM'
        input_tape_type_S3 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S3 = input_tape_type_S3.split(",")
    else:
        S3_type = "inv"
        print('Invalid response, respond as Non IBM or IBM ')
        type_input_S3 = input('Enter whether IBM or NonIBM for Server 3 ')

type_input_S4 = input('Enter whether IBM or NonIBM for Server 4 ')
type_input_server_trays_S4 = int(input('Enter how many server trays are in Server 4 '))
while S4_type == "inv":
    if type_input_S4.upper() == "NonIBM".upper() or type_input_S4.upper() == "Non IBM".upper():
        S4_type = 'NonIBM'
        input_tape_type_S4 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S4 = input_tape_type_S4.split(",")
    elif type_input_S1.upper() == "IBM".upper():
        S4_type = 'IBM'
        input_tape_type_S4 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S4 = input_tape_type_S4.split(",")
    else:
        S4_type = "inv"
        print('Invalid response, respond as Non IBM or IBM ')
        type_input_S4 = input('Enter whether IBM or NonIBM for Server 4 ')
        
type_input_S5 = input('Enter whether IBM or NonIBM for Server 5 ')
type_input_server_trays_S5 = int(input('Enter how many server trays are in Server 5 '))
while S5_type == "inv":
    if type_input_S5.upper() == "NonIBM".upper() or type_input_S5.upper() == "Non IBM".upper():
        S5_type = 'NonIBM'
        input_tape_type_S5 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S5 = input_tape_type_S5.split(",")
    elif type_input_S5.upper() == "IBM".upper():
        S5_type = 'IBM'
        input_tape_type_S5 = input('Enter what type of Tapes this server will take. Please separate with commas.')
        compatible_tapes_S5 = input_tape_type_S5.split(",")
    else:
        S5_type = "inv"
        print('Invalid response, respond as Non IBM or IBM ')
        type_input_S5 = input('Enter whether IBM or NonIBM for Server 5 ')
        
LTO6_tapes = int(input('Enter how many LTO-6 tapes are in the system. '))
LTO7_tapes = int(input('Enter how many LTO-7 tapes are in the system. '))
LTO8_tapes = int(input('Enter how many LTO-8 tapes are in the system. '))
LTO9_tapes = int(input('Enter how many LTO-9 tapes are in the system. '))

server = Tape_Simulation(S1_type, S2_type, S3_type, S4_type, S5_type, type_input_server_trays_S1, type_input_server_trays_S2, type_input_server_trays_S3, type_input_server_trays_S4, type_input_server_trays_S5, LTO6_tapes, LTO7_tapes, LTO8_tapes, LTO9_tapes, compatible_tapes_S1, compatible_tapes_S2, compatible_tapes_S3, compatible_tapes_S4, compatible_tapes_S5)

df = pd.DataFrame(columns = ['Average interarrival time (s)','Average service time Server 1 (s)','Average service time Server 2 (s)','Average service time Server 3 (s)','Average service time Server 4 (s)','Average service time Server 5 (s)','Utilization Server 1','Utilization Server 2','Utilization Server 3','Utilization Server 4','Utilization Server 5','Total Data Queued in Line Server','Total average wait time Server'])
for i in range(10): #number of replications
    np.random.seed(i)
    server.__init__(S1_type, S2_type, S3_type, S4_type, S5_type, type_input_server_trays_S1, type_input_server_trays_S2, type_input_server_trays_S3, type_input_server_trays_S4, type_input_server_trays_S5, LTO6_tapes, LTO7_tapes, LTO8_tapes, LTO9_tapes, compatible_tapes_S1, compatible_tapes_S2, compatible_tapes_S3, compatible_tapes_S4, compatible_tapes_S5)
    for j in range(server.num_tapes):
        for k in range(server.num_files):
            file_name = server.file_names[k]
            server.time_adv(j,k,file_name)
    
    if server.num_arrivals != 0:
        avg_inter_time = server.clock/server.num_arrivals
    else:
        for j in range(server.num_tapes):
            for k in range(server.num_files):
                file_name = server.file_names[k]
                server.time_adv(j,k,file_name)
    if server.num_of_departures1 != 0:
        service_time_s1 = server.dep_sum_S1/server.num_of_departures1
    else:
        service_time_s1 = 0
    if server.num_of_departures2 != 0:
        service_time_s2 = server.dep_sum_S2/server.num_of_departures2
    else:
        service_time_se = 0
    if server.num_of_departures3 != 0:
        service_time_s3 = server.dep_sum_S3/server.num_of_departures3
    else:
        service_time_s3 = 0
    if server.num_of_departures4 != 0:
        service_time_s4 = server.dep_sum_S4/server.num_of_departures4
    else:
        service_time_s4 = 0
    if server.num_of_departures5 != 0:
        service_time_s5 = server.dep_sum_S5/server.num_of_departures5
    else:
        service_time_s5 = 0

    if server.clock != 0:
        util_time_s1 = server.dep_sum_S1/server.clock
        util_time_s2 = server.dep_sum_S2/server.clock
        util_time_s3 = server.dep_sum_S3/server.clock
        util_time_s4 = server.dep_sum_S4/server.clock
        util_time_s5 = server.dep_sum_S5/server.clock
    else:
        util_time_s1 = -1
        util_time_s2 = -1
        util_time_s3 = -1
        util_time_s4 = -1
        util_time_s5 = -1
    a = pd.Series([avg_inter_time,service_time_s1,service_time_s2,service_time_s3,service_time_s4,service_time_s5,util_time_s1,util_time_s2,util_time_s3,util_time_s4,util_time_s5,server.number_in_queue,server.total_wait_time],index = df.columns)
    df = df.append(a,ignore_index = True)
    
df.to_excel('Results_Version_5_Post_Data.xlsx') 

Enter whether IBM or NonIBM for Server 1 ibm
Enter how many server trays are in Server 1 10
Enter what type of Tapes this server will take. Please separate with commas.LTO6,LTO7,LTO8,LTO9
Enter whether IBM or NonIBM for Server 2 ibm
Enter how many server trays are in Server 2 12
Enter what type of Tapes this server will take. Please separate with commas.LTO8,LTO9
Enter whether IBM or NonIBM for Server 3 nonibm
Enter how many server trays are in Server 3 10
Enter what type of Tapes this server will take. Please separate with commas.LTO6,LTO7,LTO8,LTO9
Enter whether IBM or NonIBM for Server 4 nonibm
Enter how many server trays are in Server 4 12
Enter what type of Tapes this server will take. Please separate with commas.LTO6,LTO7
Enter whether IBM or NonIBM for Server 5 nonibm
Enter how many server trays are in Server 5 10
Enter what type of Tapes this server will take. Please separate with commas.LTO8,LTO9
Enter how many LTO-6 tapes are in the system. 0
Enter how many LTO-7 tapes are in

[85, 199, 278, 366, 477, 511, 639, 728, 810, 911, 1087, 1175, 1287, 1305, 1438, 1541, 1655, 1799, 1820, 1970, 2083, 2151, 2249, 2371, 2408] [0.443928912, 0.214385019, 0.129930822, 0.065694042, 0.041573843, 0.033182024, 0.018676404, 0.013065108, 0.011390094, 0.0121271, 0.008592821, 0.004522537, 0.001658264, 0.000686756, 0.000150751, 0.000117251, 8.37507e-05, 3.35003e-05, 1.67501e-05, 3.35003e-05, 3.35003e-05, 5.02504e-05, 1.67501e-05, 1.67501e-05, 3.35003e-05]
25 25
[101, 125, 267, 380, 435, 525, 626, 705, 842, 926, 1066, 1202, 1234, 1373, 1431, 1553, 1628, 1790, 1848, 1928, 2086, 2172, 2282, 2351, 2435, 2508, 2613, 2769, 2900] [0.586803591, 0.187841227, 0.097179954, 0.055754791, 0.028586612, 0.013155882, 0.006939241, 0.005214944, 0.005497867, 0.005394638, 0.003926501, 0.002240438, 0.000860236, 0.000340271, 9.17586e-05, 1.91164e-05, 1.91164e-05, 7.64655e-06, 7.64655e-06, 1.14698e-05, 1.91164e-05, 7.64655e-06, 7.64655e-06, 1.14698e-05, 3.82327e-06, 3.82327e-06, 2.29396e-05, 1.91164e-05, 

[78, 200, 219, 314, 441, 531, 674, 742, 892, 975, 1056, 1172, 1221, 1394, 1440, 1549, 1617, 1781, 1830, 1962, 2005, 2121, 2210, 2335, 2487] [0.443928912, 0.214385019, 0.129930822, 0.065694042, 0.041573843, 0.033182024, 0.018676404, 0.013065108, 0.011390094, 0.0121271, 0.008592821, 0.004522537, 0.001658264, 0.000686756, 0.000150751, 0.000117251, 8.37507e-05, 3.35003e-05, 1.67501e-05, 3.35003e-05, 3.35003e-05, 5.02504e-05, 1.67501e-05, 1.67501e-05, 3.35003e-05]
25 25
[93, 200, 234, 305, 492, 534, 607, 798, 843, 928, 1092, 1144, 1289, 1311, 1469, 1590, 1660, 1739, 1828, 1910, 2047, 2118, 2264, 2382, 2457, 2603, 2619, 2746, 2884] [0.586803591, 0.187841227, 0.097179954, 0.055754791, 0.028586612, 0.013155882, 0.006939241, 0.005214944, 0.005497867, 0.005394638, 0.003926501, 0.002240438, 0.000860236, 0.000340271, 9.17586e-05, 1.91164e-05, 1.91164e-05, 7.64655e-06, 7.64655e-06, 1.14698e-05, 1.91164e-05, 7.64655e-06, 7.64655e-06, 1.14698e-05, 3.82327e-06, 3.82327e-06, 2.29396e-05, 1.91164e-05, 1

In [7]:
(-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 0.01)

0.0004449700405400275